#Chapter 1: Scraping OsloBors stock data

In this chapter we get our first data set from the OsloBors, Scandinavia's only independent stock exchange. 

#####The Imports
Most of the hard work in this chapter and the following chapters is done using Pandas, Pythons "library providing high-performance, easy-to-use data structures and data analysis tools". 

In [2]:
import urllib2
import StringIO
import pandas as pd
from pandas import read_html
import pymongo
from pymongo import MongoClient
import json
from datetime import datetime
from pandas.io.data import DataReader
import numpy as np

In [3]:
#For visulisations
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [4]:
#From python 3, dont have to worry about divide with floating numbers
from __future__ import division 

#####MongoDB: Where to save the data?
When we scrape stock information we want we do not want to have to rescrape it everytime we do some analysis. There are a number of methods to save the data, JSON, CSV, MYSQL, Python Pickle file, but I have decided to use MongoDB. 

MongoDB is a NoSQL database that stores information as Key:document pairs. Unlike a CSV file or equivalent, you can run powerful queries agasint MongoDB and make updates without loading and re-saving the entire file everytime.

In [4]:
#Setup MongoDB
#Switch on my file server then run in bash: mongod --dbpath /Volumes/Data/David_Files/data/db/
client = MongoClient()
db = client.oslo_bors #This is our database name
collectionStocks = db.stocks #stocks is our table (Collection) within the database

#####Creating a list of tickers
The first thing we need to know is a list of stock tickers, which we want to scrape data from. The OsloBors provides a list of all active stocks and their tickers in the form of a html table which we can get by following a given url. However, this html table is loaded via javascript, this means we cannot simply extract the html table using an html parser. The reason is because most html parsers, like beautiful soup, typically do not render javascript code. So we us a library selenium to open the url in the firefox browser, which does render javascript code. Then we extract the html from the broswer and use pandas to get extract our table and therefore our ticket list. 

The end result should look something like this.

tickers = ['ASC', 'AFG',...,'ZONC']

In [5]:
#Load the webpage in firefox and extract the content
from selenium import webdriver
browser = webdriver.Firefox()
browser.get('http://www.oslobors.no/markedsaktivitet/#/list/shares/quotelist/ob/all/false')
content = browser.page_source
browser.quit()

#Save the list of tickers
stocks = pd.read_html(content)[0]
tickers = stocks['Ticker'].tolist()

#####The link to stock data
The following links leads to statoils (STL) excel sheet containing 5 years worth of stock data. We use this link, replacing STL with each ticker in our ticker list to extract data for all our stocks.

In [18]:
#The link for the statoil stock excel 5 years
link = 'http://www.oslobors.no/ob/servlets/excel?type=history&columns=DATE%2C+CLOSE_CA%2C+BID_CA%2C+ASK_CA%2C+HIGH_CA%2C+LOW_CA%2C+TURNOVER_TOTAL%2C+VOLUME_TOTAL_CA%2C+TRADES_COUNT%2C+TRADES_COUNT_TOTAL%2C+VWAP&format[DATE]=ddd.mm.YY&format[CLOSE_CA]=%23%2C%23%230.00%23%23%23&format[BID_CA]=%23%2C%23%230.00%23%23&format[ASK_CA]=%23%2C%23%230.00%23%23&format[HIGH_CA]=%23%2C%23%230.00%23%23%23&format[LOW_CA]=%23%2C%23%230.00%23%23%23&format[TURNOVER_TOTAL]=%23%2C%23%230&format[VOLUME_TOTAL_CA]=%23%2C%23%230&format[TRADES_COUNT]=%23%2C%23%230&format[TRADES_COUNT_TOTAL]=%23%2C%23%230&format[VWAP]=%23%2C%23%230.00%23%23%23&header[DATE]=STL&header[CLOSE_CA]=Last&header[BID_CA]=Bid&header[ASK_CA]=Ask&header[HIGH_CA]=High&header[LOW_CA]=Low&header[TURNOVER_TOTAL]=Total%20turnover%20%28NOK%29&header[VOLUME_TOTAL_CA]=Total%20amount%20shares%20volume&header[TRADES_COUNT]=Amount%20off.%20trades&header[TRADES_COUNT_TOTAL]=Amount%20trades%20total&header[VWAP]=VWAP&view=DELAYED&source=feed.ose.quotes.INSTRUMENTS&filter=ITEM_SECTOR%3D%3DsSTL.OSE%26%26DELETED!%3Dn1&stop=now&start=1286402400000&space=DAY&ascending=true&limit=10000000&filename=data.xlsx'

#####Downloading the stock data
The following code simply runs a loop over all the tickers in our ticker list, puts them in the statoil URL link and downloads the excel file as a pandas dataframe.

In [20]:
collectionStocks.remove() #Remove the old data from the MongoDB database

for i in tickers:
    
    print str(i) + " - Scraping"
    
    #Extract the Data from the excel
    ticker = str(i).replace(" ","%20") #Some tickers have spaces in their names ('SAS NOK'), so we replace the space with a %20 to not break the URL
    url = link.replace("STL", ticker) #Swap out statoils ticker for the one in our list
    xld = urllib2.urlopen(url).read()
    xlds = StringIO.StringIO(xld)
    data = pd.read_excel(xlds, "data") #Turn the excel file into a pandas dataframe
    
    data.columns = data.columns.str.replace('.','') #The column names are keys in mongoDB and mongoDB doesnt like '.' in its key names
    
    data.rename(columns = {str(i):'Date'}, inplace=True) #Name the date column to date, it is orginally called after the ticker
    
    data['Ticker'] = str(i) #Add a ticker column
    
    records = json.loads(data.T.to_json(date_format='iso')).values() #Convert dataframe to json, which is a format that can be directly loaded into mongoDB
    
    #Here we check for any errors before saving the data to mongoDB
    if len(records) > 0:
        collectionStocks.insert(records) #Store the data in the mongoDB
        
        print str(i) + " - Added to the database" #Current stock we are scraping
    else:
        print str(i) + " - Not scraped, see comment" #some tickers use OAX and not OSE in its link string, so our statoil link returns empty for these tickers. For now we just ignore them
        

ASC - Scraping
ASC - Added to the database
...ZONC - Scraping
ZONC - Not scraped, see comment


#####Test if our data saved correctly
Just to make sure mongoDB was working as I thought it should, I extract a signle row from the database and load it into a pandas dataframe.

In [53]:
test_data = collectionStocks.find_one() #Find a single value in our mongoDB database and check it isn't empty
test = pd.DataFrame.from_dict([test_data]) #Load that data into a pandas dataframe
#print test.columns.values[2].encode(encoding='latin-1')

#####Finished
We now have 5 years of OsloBors data for most of our ticker list saved into our database. The next step is to get a basic understanding of our data by doing some stock analysis.